In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import log
import seaborn as sb

In [3]:
truth=pd.read_table('/mnt/scratch2/cedar/input_files/i100_taxonomy.truth')
truth = truth.groupby('Species').sum().reset_index()
puff=pd.read_table('/mnt/scratch2/cedar/results/script_results/i100_31_sr1_puff/out.tab')
puff = puff.groupby('Species').sum().reset_index()
salmon=pd.read_table('/mnt/scratch2/cedar/results/script_results/i100_31_sr1_salmon/out.tab')
salmon = salmon.groupby('Species').sum().reset_index()
cedar=pd.read_table('/mnt/scratch2/cedar/results/script_results/i100_31_sr1_cedar/out.tab')
cedar = cedar.groupby('Species').sum().reset_index()

In [6]:
al=pd.merge(truth, puff, on='Species', how='outer')[['Species', 'NumCounts_x', 'NumCounts_y']].fillna(0)
al.columns = ['Species', 'truth', 'puff']
al=pd.merge(al, salmon, on='Species', how='outer')[['Species', 'truth', 'puff', 'NumCounts']].fillna(0)
al.columns = ['Species', 'truth', 'puff', 'salmon']
al=pd.merge(al, cedar, on='Species', how='outer')[['Species', 'truth', 'puff', 'salmon', 'NumCounts']].fillna(0)
al.columns = ['Species', 'truth', 'puff', 'salmon', 'cedar']

In [5]:
al

,Species,truth,puff,salmon,cedar
0,Acinetobacter baumannii,262164.0,549.513,2184.705,6958.0
1,Alkalilimnicola ehrlichii,230160.0,509.162,2165.511,528.0
2,Alkaliphilus metalliredigens,374667.0,781.633,3245.585,990.0
3,Bacillus anthracis,408054.0,511.744,3667.453,2804.0
4,Bacillus cereus,1320777.0,2352.731,11637.614,7408.0
5,Bacillus clausii,286614.0,576.886,2582.661,742.0
6,Bacillus halodurans,274294.0,589.009,2407.603,918.0
7,Bacillus subtilis,272560.0,600.297,2461.188,808.0
8,Bacillus thuringiensis,372187.0,1461.580,3265.563,3364.0
9,Bacteroides thetaiotaomicron,402214.0,796.490,3396.130,893.0
